In [1]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import pyspark

number_cores = 8
memory_gb = 24
conf = (
    pyspark.SparkConf()
        .setMaster('local[{}]'.format(number_cores))
        .set('spark.driver.memory', '{}g'.format(memory_gb))
)

In [6]:
conda install -c conda-forge wordcloud

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda/envs/Python3

  added / updated specs:
    - wordcloud


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    wordcloud-1.6.0            |   py36h516909a_0         184 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

  wordcloud          conda-forge/linux-64::wordcloud-1.6.0-py36h516909a_0

The following packages will be SUPERSEDED by a higher-priority channel:

  ca-certificates     pkgs/main::ca-certificates-2020.1.1-0 --> conda-forge:

In [20]:
!pip install snorkel

In [21]:
import pyspark
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from pyspark.sql import SQLContext
from wordcloud import WordCloud

In [22]:
# from textblob import TextBlob
from snorkel.labeling import labeling_function, labeling_function, LabelModel, PandasLFApplier
from snorkel.labeling.apply.spark import SparkLFApplier

ImportError: cannot import name 'to_int_label_array'

In [9]:
sc = pyspark.SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [10]:
# Load the Data

# Make a subset of our dataframe using pandas and keep only the columns we need, remove rows with NaN
# Ideally we need to do this part done using Spark
amazon_reviews_dataset_pd_df = pd.read_csv("/project/DE_Group_Project_2020/Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv")
amazon_reviews_dataset_pd_df = pd.concat([amazon_reviews_dataset_pd_df['reviews.rating'],amazon_reviews_dataset_pd_df['reviews.text']], axis=1)
amazon_reviews_dataset_pd_df = amazon_reviews_dataset_pd_df.dropna()
amazon_reviews_dataset_pd_df.rename(columns={"reviews.rating": "reviews_rating", "reviews.text": "reviews_text"}, inplace=True)
amazon_reviews_dataset_pd_df

# Convert pandas DF to spark dataframe
amazon_reviews_dataset_sql_context = sqlContext.createDataFrame(amazon_reviews_dataset_pd_df)

In [11]:
# Temp view for maniputing the data using SQL queries
amazon_reviews_dataset_sql_context.createOrReplaceTempView('amazon_reviews')

# Spliting the review text by review level, these will be the basis for learning information about the data that
# can be used for creating heuristics for our labeling functions. 

amazon_reviews_level1 = sqlContext.sql("SELECT reviews_text FROM amazon_reviews where reviews_rating == 1")
amazon_reviews_level2 = sqlContext.sql("SELECT reviews_text FROM amazon_reviews where reviews_rating == 2")
amazon_reviews_level3 = sqlContext.sql("SELECT reviews_text FROM amazon_reviews where reviews_rating == 3")
amazon_reviews_level4 = sqlContext.sql("SELECT reviews_text FROM amazon_reviews where reviews_rating == 4")
amazon_reviews_level5 = sqlContext.sql("SELECT reviews_text FROM amazon_reviews where reviews_rating == 5")


In [12]:
Y_dev = amazon_reviews_level1.sentiment.values

AttributeError: 'DataFrame' object has no attribute 'sentiment'

In [17]:
# Visualise the 5 different reviews WordCloud and figure other ways to quantify frequency of words
# From - https://towardsdatascience.com/spam-classifier-in-python-from-scratch-27a98ddd8e73

# WorldCloud - Review == 1 
reviewone = ' '.join(list(amazon_reviews_level1)
reviewone_wc = WordCloud(width=512,height=512).generate(reviewone)

plt.figure(figsize = (10,8), facecolor = 'k')
plt.imshow(reviewone_wc)
plt.axis('off')
plt.tight_layout(pad = 0)
plt.show()

SyntaxError: invalid syntax (<ipython-input-17-6210b7207b2c>, line 6)

In [18]:
# Set voting values.
RATINGONE = 0
RATINGTWO = 1 
RATINGTHREE = 2
RATINGFOUR = 3
RATINGFIVE = 4

ABSTAIN = -1

In [ ]:
from textblob import TextBlob 
from snorkel.preprocess import preprocessor

# Common good words 
GOOD = r"\bjew (love|great|good|easy|happy|nice|fun|pretty|works|better|well|ok)"
BAD = r"\bjew (old|junk|useless|failded|slow|nothing)"

# labelling functions for five different rating tiers 
def good(x):
  if re.search(GOOD, x) else ABSTAIN
     return RATINGTHREE if TextBlob() else ABSTAIN
     return RATINGFOUR if  else ABSTAIN
     return RATINGFIVE if  else ABSTAIN
  if re.search(BAD, x) else ABSTAIN
     return RATINGTWO if  else ABSTAIN
     return RATINGTHRE if  else ABSTAIN